# Automated ML

Import Dependencies. In the cell below, import all the dependencies that are needed to complete the project.

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import datasets
import logging
import json
import requests
import os

# !pip install kaggle

# import kaggle

import azureml.core
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails

from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig

import joblib

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.49.0


### Initialization

Initialize a workspace object from persisted configuration

In [4]:
ws = Workspace.from_config()
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'AutoML_experiment'
project_folder = './training'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

# Choose a name for your CPU cluster
cluster_name = "compute-cluster-p3"

# Create compute cluster
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('An existing cluster will be used!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)
    print('An new cluster will be created now!')

aml_compute.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

quick-starts-ws-231701
aml-quickstarts-231701
westeurope
b968fb36-f06a-4c76-a15f-afab68ae7667
An existing cluster will be used!
Succeeded......................

### Data
The dataset for heart failure prediction was taken from kaggle and is saved locally. The code first tries to use that and if not reachable extract the data from the provided link.

The data includes several columns, namely:
`age`, `anaemia`, `creatinine_phosphokinase`, `diabetes`, `ejection_fraction`, `high_blood_pressure`, `platelets`, `serum_creainine`, `serum_sodium`, `sex`, `smoking`, `time` and `DEATH_EVENT`.

The patients `DEATH_EVENT` will be predicted based on the the other parameters listed in the dataset.

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# update the key to match the dataset name
found = False
key = "Experiment"
description_text = "Heart Failure Prediction - Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        url = "https://raw.githubusercontent.com/mirsadraee/Udacity_ND_Azure_Machine_Learning_Projects/develop/Project_03/heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head(10)

In [ ]:
df['age'].plot.hist(bins=20, alpha=0.5)

In [ ]:
df['smoking'].plot.hist(bins=3, alpha=0.5)

In [ ]:
df['sex'].plot.hist(bins=3, alpha=0.5)

In [ ]:
df['DEATH_EVENT'].plot.hist(bins=3, alpha=0.5)

In [ ]:
df['time'].plot.hist(bins=50, alpha=0.5)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# Automl settings
automl_settings = {"n_cross_validations": 2,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
                    }

# Parameters for AutoMLConfig
automl_config = AutoMLConfig(compute_target = aml_compute,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            **automl_settings
                            )

In [ ]:
# Submit the experiment
remote_run = experiment.submit(automl_config, show_output = True)
remote_run.wait_for_completion()

## Run Details
To use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run, fitted_automl_model = remote_run.get_output()

best_automl_run

In [ ]:
best_automl_run.register_model(model_name = "AutoML_best_run.pkl", model_path = './outputs/')

print(best_automl_run)


In [ ]:
# Save the best model
joblib.dump(fitted_automl_model, 'outputs/AutoML_Model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below,  following steps are taken place:
- register the model, 
- create an inference config and 
- deploy the model as a web service.

In [ ]:
# register the model
model = remote_run.register_model(model_name = "AutoML_Model")

# deploy the model
deploy_config = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=2,enable_app_insights=True)

# configure the inference
# An inference config in Azure Machine Learning is used to define the environment and entry script required to deploy a model as a web service.
inference_config = InferenceConfig(entry_script='./score.py')

service_name = 'automl-web-service'
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=deploy_config)
service.wait_for_deployment(True)

print(service.state)

In the cell below, a request is sent to the web service to test it.

In [ ]:
scoring_uri = service.scoring_uri

headers = {'Content-Type':'application/json'}

test_data_1 = json.dumps({'data':[{
    'age':75,
    'anaemia':0,
    'creatinine_phosphokinase':582,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':265000,
    'serum_creatinine':1.9,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

test_data_2 = json.dumps({'data':[{
    'age':40,
    'anaemia':0,
    'creatinine_phosphokinase':321,
    'diabetes':0,
    'ejection_fraction':35,
    'high_blood_pressure':0,
    'platelets':265000,
    'serum_creatinine':1,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':198}
    ]
        })

response = requests.post(scoring_uri, data=test_data_1, headers=headers)
print("Result 1:",response.text)


response = requests.post(scoring_uri, data=test_data_2, headers=headers)
print("Result 2:",response.text)

The logs of the web service are printed.

In [ ]:
# service.get_logs()

The service will be deleted now.

In [ ]:
# service.delete()

**Submission Checklist**
- [x]  I have registered the model.
- [x]  I have deployed the model with the best accuracy as a webservice.
- [x]  I have tested the webservice by sending a request to the model endpoint.
- [x]  I have deleted the webservice and shutdown all the computes that I have used.
- [x]  I have taken a screenshot showing the model endpoint as active.
- [x]  The project includes a file containing the environment details.
